<a href="https://colab.research.google.com/github/Anirudh-Kannan/VGG-Animal-Classification/blob/master/VGG_animal_classification_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.applications.vgg16 import VGG16

vgg = VGG16()
model = Sequential()


for i in vgg.layers:
    model.add(i)

model.summary()

for i in model.layers:
    i.trainable = False

model.pop()
model.add(Dense(3, activation='softmax'))

model.summary()

import cv2


class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.inter = inter
        
    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation = self.inter)

import os
import numpy as np
import cv2


class SimpleDatasetLoader:

    def __init__(self, preprocessors = None):
        self.preprocessors = preprocessors

        if preprocessors is None:
            self.preprocessors = []

    
    def load(self, imgpaths, verbose = -1):
        data=[]
        labels=[]


        for (i, imgpath) in enumerate(imgpaths):
            image = cv2.imread(imgpath)
            label = imgpath.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)


            if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
                print("[INFO] processed {}/{}".format(i + 1, len(imgpaths)))

        
        return (np.array(data),np.array(labels))

Instructions for updating:
Colocations handled automatically by placer.
553467904/553467096 [==============================] - 17s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__

In [0]:
!pip install imutils

In [0]:
from imutils import paths

image_paths = list(paths.list_images('/content/gdrive/My Drive/animals'))

In [0]:
sp = SimplePreprocessor(224, 224)
sdl = SimpleDatasetLoader(preprocessors=[sp])
(data, labels) = sdl.load(image_paths, verbose=500)

####Time taken to preprocess is very high.
An alternative is explored in the next version